# Variational Autoencoders & Real-Time Streaming Anomaly Detection

From the [Sisyphean Gridworks ML Playground](https://sgridworks.com/ml-playground/guides/16-advanced-anomaly-detection.html)

## Setup

Clone the repository and install dependencies. Run this cell first.

In [ ]:
!git clone https://github.com/SGridworks/Dynamic-Network-Model.git 2>/dev/null || echo 'Already cloned'
%cd Dynamic-Network-Model
!pip install -q pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm pyarrow

## From Autoencoder to VAE — Why Probabilistic?

In Guide 08, the basic autoencoder compressed voltage features into a fixed latent vector and reconstructed them. Anomalies had high reconstruction error. This works, but the latent space is unstructured—there is no guarantee that nearby latent points represent similar voltage patterns, and there is no way to quantify how unlikely a given reading is.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_recall_curve, average_precision_score

from demo_data.load_demo_data import load_customer_interval_data, load_weather_data, load_outage_history

# Load AMI data (same as Guide 08)
ami = load_customer_interval_data()
ami["timestamp"] = pd.to_datetime(ami["timestamp"])

# Aggregate to hourly statistics per customer (same features as Guide 08)
hourly = ami.groupby(["customer_id", ami["timestamp"].dt.floor("h")]).agg(
    voltage_mean=("voltage_v", "mean"),
    voltage_std=("voltage_v", "std"),
    voltage_min=("voltage_v", "min"),
    voltage_max=("voltage_v", "max"),
    energy_kwh=("energy_kwh", "sum"),
).reset_index()
hourly["voltage_range"] = hourly["voltage_max"] - hourly["voltage_min"]
hourly = hourly.fillna(0)

feature_cols = ["voltage_mean", "voltage_std", "voltage_range",
                "voltage_min", "voltage_max", "energy_kwh"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(hourly[feature_cols])

print(f"Hourly feature rows: {len(hourly):,}")
print(f"Features: {feature_cols}")

## Build the VAE Architecture

The VAE has three key differences from the basic autoencoder in Guide 08:

In [ ]:
class VoltageVAE(nn.Module):
    def __init__(self, input_dim, latent_dim=3):
        super().__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # Encoder: shared layers
        self.encoder_shared = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
        )
        # Two separate heads: one for mu, one for log_var
        self.fc_mu      = nn.Linear(16, latent_dim)
        self.fc_log_var = nn.Linear(16, latent_dim)

        # Decoder: reconstruct from latent sample
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
        )

    def encode(self, x):
        """Return mu and log_var for the latent distribution."""
        h = self.encoder_shared(x)
        mu      = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        """Sample z using the reparameterization trick."""
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)       # sample epsilon ~ N(0, 1)
        z   = mu + std * eps              # shift and scale
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var

# Define the VAE loss function
def vae_loss(x, x_recon, mu, log_var):
    """
    ELBO loss = Reconstruction loss + KL divergence.
    Returns total loss and both components separately for monitoring.
    """
    # Reconstruction loss: mean squared error per sample
    recon_loss = F.mse_loss(x_recon, x, reduction="mean")

    # KL divergence: D_KL(q(z|x) || p(z)) where p(z) = N(0, I)
    # Closed-form for two Gaussians:
    #   -0.5 * sum(1 + log_var - mu^2 - exp(log_var))
    kl_loss = -0.5 * torch.mean(
        torch.sum(1 + log_var - mu.pow(2) - log_var.exp(), dim=1)
    )

    total_loss = recon_loss + kl_loss
    return total_loss, recon_loss, kl_loss

# Instantiate the model
input_dim = len(feature_cols)
vae = VoltageVAE(input_dim, latent_dim=3)
print(vae)
print(f"\nTotal parameters: {sum(p.numel() for p in vae.parameters()):,}")

## Train the VAE on Normal Voltage Data

Following the same approach as Guide 08, we train exclusively on normal data. We first run an Isolation Forest to identify and exclude obvious anomalies from the training set, then train the VAE to learn the distribution of normal voltage patterns.

In [ ]:
# Pre-filter: remove obvious anomalies with Isolation Forest (same as Guide 08)
iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42)
iso_forest.fit(X_scaled)
iso_labels = iso_forest.predict(X_scaled)

# Keep only normal data for VAE training
normal_mask = iso_labels == 1
X_normal = X_scaled[normal_mask]
print(f"Training samples (normal only): {len(X_normal):,}")

# Train/validation split (80/20)
split = int(len(X_normal) * 0.8)
train_tensor = torch.FloatTensor(X_normal[:split])
val_tensor   = torch.FloatTensor(X_normal[split:])

train_loader = DataLoader(
    TensorDataset(train_tensor, train_tensor),
    batch_size=128, shuffle=True
)

# Training setup
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)

# Track losses separately
history = {"total": [], "recon": [], "kl": []}

# Train for 80 epochs
for epoch in range(80):
    vae.train()
    epoch_total, epoch_recon, epoch_kl = 0, 0, 0

    for batch_x, _ in train_loader:
        x_recon, mu, log_var = vae(batch_x)
        total, recon, kl = vae_loss(batch_x, x_recon, mu, log_var)

        optimizer.zero_grad()
        total.backward()
        optimizer.step()

        epoch_total += total.item()
        epoch_recon += recon.item()
        epoch_kl    += kl.item()

    n_batches = len(train_loader)
    history["total"].append(epoch_total / n_batches)
    history["recon"].append(epoch_recon / n_batches)
    history["kl"].append(epoch_kl / n_batches)

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:>3}/80  "
              f"Total: {history['total'][-1]:.4f}  "
              f"Recon: {history['recon'][-1]:.4f}  "
              f"KL: {history['kl'][-1]:.4f}")

## Compute Probabilistic Anomaly Scores with the ELBO

The basic autoencoder from Guide 08 uses reconstruction error as its anomaly score. The VAE gives us a richer signal: the Evidence Lower Bound (ELBO), which combines reconstruction probability with the KL divergence. A low ELBO means the data point is unlikely under the learned model—it is both hard to reconstruct and its latent encoding is far from the prior distribution.

In [ ]:
# Plot training curves: reconstruction loss and KL divergence separately
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history["recon"], color="#5FCCDB", label="Reconstruction Loss")
ax1.plot(history["kl"], color="#D69E2E", label="KL Divergence")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("VAE Training: Loss Components")
ax1.legend()

ax2.plot(history["total"], color="#2D6A7A", linewidth=2)
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Total ELBO Loss")
ax2.set_title("VAE Training: Total Loss (ELBO)")

plt.tight_layout()
plt.show()

## Compare VAE vs Autoencoder vs Isolation Forest

To fairly compare all three approaches, we need ground-truth labels that are independent of any model. Using one model’s output (e.g., Isolation Forest flags) as pseudo-ground-truth to evaluate that same model would be circular and methodologically invalid. Instead, we inject synthetic anomalies—voltage sags, spikes, and drift—into a held-out portion of the data at known timestamps. This gives us an objective, model-independent ground truth for precision-recall evaluation.

In [ ]:
# Score ALL data points (normal + anomalous) through the VAE
vae.eval()
all_tensor = torch.FloatTensor(X_scaled)

with torch.no_grad():
    x_recon, mu, log_var = vae(all_tensor)

    # Per-sample reconstruction error
    recon_error = torch.mean((all_tensor - x_recon) ** 2, dim=1)

    # Per-sample KL divergence
    kl_per_sample = -0.5 * torch.sum(
        1 + log_var - mu.pow(2) - log_var.exp(), dim=1
    )

    # Combined ELBO anomaly score (higher = more anomalous)
    elbo_score = recon_error + kl_per_sample

hourly["vae_recon_error"] = recon_error.numpy()
hourly["vae_kl_div"]      = kl_per_sample.numpy()
hourly["vae_elbo_score"]  = elbo_score.numpy()

# Set threshold at 99th percentile of ELBO score
elbo_threshold = hourly["vae_elbo_score"].quantile(0.99)
hourly["vae_anomaly"] = (hourly["vae_elbo_score"] > elbo_threshold).astype(int)

print(f"ELBO threshold (99th pctile): {elbo_threshold:.4f}")
print(f"VAE anomalies detected:       {hourly['vae_anomaly'].sum()}")
print(f"\nScore components (anomalies only):")
anomalous = hourly[hourly["vae_anomaly"] == 1]
print(f"  Avg reconstruction error: {anomalous['vae_recon_error'].mean():.4f}")
print(f"  Avg KL divergence:        {anomalous['vae_kl_div'].mean():.4f}")

## Design a Real-Time Detection Pipeline

Everything so far has been batch detection: we load all the data, train a model, and score everything at once. In a real utility control room, AMI data arrives continuously—every 15 minutes from thousands of customers. We need a system that processes data as it arrives, in sliding windows, and flags anomalies in near-real-time.

In [ ]:
# Visualize the two components of the anomaly score
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scatter: reconstruction error vs KL divergence
normal_pts = hourly[hourly["vae_anomaly"] == 0]
anomaly_pts = hourly[hourly["vae_anomaly"] == 1]

ax1.scatter(normal_pts["vae_recon_error"], normal_pts["vae_kl_div"],
            c="#5FCCDB", s=3, alpha=0.2, label="Normal")
ax1.scatter(anomaly_pts["vae_recon_error"], anomaly_pts["vae_kl_div"],
            c="red", s=25, marker="x", label="Anomaly")
ax1.set_xlabel("Reconstruction Error")
ax1.set_ylabel("KL Divergence")
ax1.set_title("VAE Anomaly Score Components")
ax1.legend()

# ELBO score distribution
ax2.hist(hourly["vae_elbo_score"], bins=150, color="#5FCCDB", edgecolor="white")
ax2.axvline(x=elbo_threshold, color="red", linestyle="--",
           label=f"Threshold ({elbo_threshold:.3f})")
ax2.set_xlabel("ELBO Anomaly Score")
ax2.set_ylabel("Frequency")
ax2.set_title("ELBO Score Distribution")
ax2.set_yscale("log")
ax2.legend()

plt.tight_layout()
plt.show()

## Implement the Sliding Window Processor

The core of the streaming pipeline is a class that maintains a buffer of recent readings per customer, computes features when the window is full, and scores each window through the trained VAE.

In [ ]:
# --- Inject synthetic anomalies as model-independent ground truth ---
np.random.seed(42)
n_total = len(X_scaled)
y_true = np.zeros(n_total, dtype=int)

# Select 2% of indices to inject anomalies
n_inject = int(n_total * 0.02)
inject_idx = np.random.choice(n_total, size=n_inject, replace=False)
y_true[inject_idx] = 1

# Create a modified copy for scoring (leave training data unchanged)
X_eval = X_scaled.copy()

# Three anomaly types with known signatures:
for i, idx in enumerate(inject_idx):
    anomaly_type = i % 3
    if anomaly_type == 0:  # voltage sag: mean drops 3+ std
        X_eval[idx, 0] -= np.random.uniform(3.0, 5.0)
    elif anomaly_type == 1:  # voltage spike: mean rises 3+ std
        X_eval[idx, 0] += np.random.uniform(3.0, 5.0)
    else:  # high variance: range and std increase
        X_eval[idx, 1] += np.random.uniform(3.0, 6.0)  # voltage_std
        X_eval[idx, 2] += np.random.uniform(3.0, 6.0)  # voltage_range

eval_tensor = torch.FloatTensor(X_eval)
print(f"Injected {n_inject} synthetic anomalies ({n_inject/n_total*100:.1f}%)")
print(f"  Types: {n_inject//3} sags, {n_inject//3} spikes, {n_inject - 2*(n_inject//3)} high-variance")

# --- Train basic autoencoder (matched architecture to VAE for fair comparison) ---
# AE bottleneck = 3 dimensions (same as VAE latent_dim) to control for capacity
class BasicAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32), nn.ReLU(),
            nn.Linear(32, 16),  nn.ReLU(),
            nn.Linear(16, 3),
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 16),  nn.ReLU(),
            nn.Linear(16, 32), nn.ReLU(),
            nn.Linear(32, input_dim),
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

ae = BasicAutoencoder(input_dim)
ae_optimizer = torch.optim.Adam(ae.parameters(), lr=0.001)
ae_criterion = nn.MSELoss()

# Train basic AE for 50 epochs on same normal data
ae_loader = DataLoader(TensorDataset(train_tensor, train_tensor),
                        batch_size=128, shuffle=True)
for epoch in range(50):
    ae.train()
    for bx, by in ae_loader:
        loss = ae_criterion(ae(bx), by)
        ae_optimizer.zero_grad()
        loss.backward()
        ae_optimizer.step()

# Score with basic autoencoder on synthetic-injected data
ae.eval()
with torch.no_grad():
    ae_recon = ae(eval_tensor)
    ae_error = torch.mean((eval_tensor - ae_recon) ** 2, dim=1).numpy()

# Score with Isolation Forest on synthetic-injected data
iso_scores = -iso_forest.decision_function(X_eval)  # negate so higher = more anomalous

# Score with VAE on synthetic-injected data
vae.eval()
with torch.no_grad():
    mu, log_var = vae.encode(eval_tensor)
    z = vae.reparameterize(mu, log_var)
    recon = vae.decode(z)
    recon_err = torch.mean((eval_tensor - recon) ** 2, dim=1)
    kl_div = -0.5 * torch.sum(1 + log_var - mu**2 - log_var.exp(), dim=1)
    vae_elbo_scores = (recon_err + kl_div).numpy()

# Ground truth: synthetic anomaly labels (y_true defined above)

# Precision-recall curves for each method
fig, ax = plt.subplots(figsize=(10, 7))

methods = {
    "Isolation Forest":    iso_scores,
    "Basic Autoencoder":   ae_error,
    "VAE (ELBO Score)":    vae_elbo_scores,
}
colors = ["#718096", "#D69E2E", "#2D6A7A"]

for (name, scores), color in zip(methods.items(), colors):
    precision, recall, _ = precision_recall_curve(y_true, scores)
    ap = average_precision_score(y_true, scores)
    ax.plot(recall, precision, label=f"{name} (AP={ap:.3f})",
            color=color, linewidth=2)

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Precision-Recall: Anomaly Detection Methods Compared")
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Add Adaptive Thresholding

A fixed threshold works in stable conditions, but voltage patterns change with the seasons. In summer, air conditioning loads cause higher voltage variance—what looks anomalous in January might be perfectly normal in July. An adaptive threshold uses a rolling percentile of recent scores to set the alert level, automatically adjusting to seasonal patterns.

In [ ]:
# Score distributions: normal vs anomalous for each method
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (name, scores) in zip(axes, methods.items()):
    normal_scores  = scores[y_true == 0]
    anomaly_scores = scores[y_true == 1]

    ax.hist(normal_scores, bins=80, alpha=0.6, color="#5FCCDB",
           label="Normal", density=True)
    ax.hist(anomaly_scores, bins=30, alpha=0.7, color="red",
           label="Anomaly", density=True)
    ax.set_title(name)
    ax.set_xlabel("Anomaly Score")
    ax.set_ylabel("Density")
    ax.legend()

plt.suptitle("Score Distributions: Normal vs Anomalous Points", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Simulate Real-Time Operation

Now we replay one week of AMI data through the streaming pipeline, simulating what would happen if this system were running in SP&L's control room. We then overlay actual outage events to see how our detections correlate with real grid problems.

In [ ]:
# Load weather data for seasonal context
weather = load_weather_data()

# Load outage events for validation later
outages = load_outage_history()

print(f"Weather records:  {len(weather):,}")
print(f"Outage events:   {len(outages):,}")
print(f"Customers:       {ami['customer_id'].nunique():,}")

## Model Persistence and Hyperparameter Justification

For deployment, you need to save both the VAE weights and the scaler that was fit on training data. Without the scaler, new incoming data cannot be normalized consistently.

In [ ]:
from collections import defaultdict, deque

class StreamingAnomalyDetector:
    """
    Sliding window anomaly detector for AMI voltage data.
    Maintains a 1-hour buffer per customer and scores with a trained VAE.
    """
    def __init__(self, vae_model, scaler, feature_cols,
                 window_size=4, threshold=None):
        """
        Args:
            vae_model:    trained VoltageVAE instance
            scaler:       fitted StandardScaler
            feature_cols: list of feature column names
            window_size:  number of readings per window (4 = 1 hour at 15-min intervals)
            threshold:    fixed ELBO threshold (None = use adaptive)
        """
        self.vae = vae_model
        self.vae.eval()
        self.scaler = scaler
        self.feature_cols = feature_cols
        self.window_size = window_size
        self.fixed_threshold = threshold

        # Per-customer sliding window buffers
        self.buffers = defaultdict(
            lambda: deque(maxlen=window_size)
        )

        # Detection log
        self.detections = []
        self.all_scores = []

    def ingest(self, customer_id, timestamp, voltage, energy_kwh):
        """Process a single AMI reading."""
        self.buffers[customer_id].append({
            "timestamp": timestamp,
            "voltage_v": voltage,
            "energy_kwh": energy_kwh,
        })

        # Only score when we have a full window
        if len(self.buffers[customer_id]) return None

        # Extract features from the window
        features = self._extract_features(customer_id)
        score = self._score(features)

        self.all_scores.append({
            "customer_id": customer_id,
            "timestamp": timestamp,
            "elbo_score": score,
        })

        return score

    def _extract_features(self, customer_id):
        """Compute hourly statistics from the sliding window."""
        readings = list(self.buffers[customer_id])
        voltages = np.array([r["voltage_v"] for r in readings])
        consumption = sum(r["energy_kwh"] for r in readings)

        return {
            "voltage_mean":     voltages.mean(),
            "voltage_std":      voltages.std(),
            "voltage_range":    voltages.max() - voltages.min(),
            "voltage_min":      voltages.min(),
            "voltage_max":      voltages.max(),
            "energy_kwh":  consumption,
        }

    def _score(self, features):
        """Run features through the VAE and return ELBO score."""
        x = np.array([[features[c] for c in self.feature_cols]])
        x_scaled = self.scaler.transform(x)
        x_tensor = torch.FloatTensor(x_scaled)

        with torch.no_grad():
            x_recon, mu, log_var = self.vae(x_tensor)
            recon = F.mse_loss(x_recon, x_tensor, reduction="none").mean(dim=1)
            kl = -0.5 * torch.sum(
                1 + log_var - mu.pow(2) - log_var.exp(), dim=1
            )
            score = (recon + kl).item()

        return score

# Create the detector with our trained VAE
detector = StreamingAnomalyDetector(
    vae_model=vae,
    scaler=scaler,
    feature_cols=feature_cols,
    window_size=4,
    threshold=elbo_threshold,
)

print("Streaming detector initialized.")
print(f"  Window size:     {detector.window_size} readings (1 hour)")
print(f"  Fixed threshold: {elbo_threshold:.4f}")

## What You Built and Next Steps

In [ ]:
class AdaptiveThreshold:
    """
    Rolling percentile threshold that adapts to seasonal patterns.
    Maintains a window of recent scores and computes the threshold
    as the Nth percentile of that window.
    """
    def __init__(self, window_hours=168, percentile=99.0,
                 min_samples=24, fallback_threshold=1.0):
        """
        Args:
            window_hours:      lookback window for computing percentile (168 = 1 week)
            percentile:        percentile to use as threshold (99 = flag top 1%)
            min_samples:       minimum scores before adaptive threshold activates
            fallback_threshold: fixed threshold used until min_samples is reached
        """
        self.window_size = window_hours
        self.percentile = percentile
        self.min_samples = min_samples
        self.fallback = fallback_threshold

        self.score_history = deque(maxlen=window_hours)

    def update_and_check(self, score):
        """Add a score and return (is_anomaly, current_threshold)."""
        self.score_history.append(score)

        if len(self.score_history) else:
            threshold = np.percentile(
                list(self.score_history), self.percentile
            )

        is_anomaly = score > threshold
        return is_anomaly, threshold

# Upgrade the detector with adaptive thresholding
class AdaptiveStreamingDetector(StreamingAnomalyDetector):
    """Streaming detector with per-customer adaptive thresholds."""

    def __init__(self, vae_model, scaler, feature_cols,
                 window_size=4, lookback_hours=168, percentile=99.0):
        super().__init__(vae_model, scaler, feature_cols, window_size)
        self.lookback_hours = lookback_hours
        self.percentile = percentile

        # Per-customer adaptive thresholds
        self.adaptive_thresholds = defaultdict(
            lambda: AdaptiveThreshold(
                window_hours=lookback_hours,
                percentile=percentile,
                fallback_threshold=elbo_threshold,
            )
        )

    def ingest(self, customer_id, timestamp, voltage, energy_kwh):
        """Process a reading with adaptive thresholding."""
        score = super().ingest(customer_id, timestamp, voltage, energy_kwh)

        if score is None:
            return None

        # Check against adaptive threshold for this customer
        is_anomaly, threshold = self.adaptive_thresholds[customer_id].update_and_check(score)

        if is_anomaly:
            self.detections.append({
                "customer_id":   customer_id,
                "timestamp":  timestamp,
                "elbo_score": score,
                "threshold":  threshold,
            })

        return {"score": score, "threshold": threshold, "is_anomaly": is_anomaly}

# Create adaptive detector
adaptive_detector = AdaptiveStreamingDetector(
    vae_model=vae,
    scaler=scaler,
    feature_cols=feature_cols,
    window_size=4,
    lookback_hours=168,    # 1-week rolling window
    percentile=99.0,       # flag top 1%
)

print("Adaptive streaming detector ready.")
print(f"  Lookback window: {adaptive_detector.lookback_hours} hours (1 week)")
print(f"  Percentile:      {adaptive_detector.percentile}th")

In [ ]:
# Select one week of data for simulation (matches the AMI data range)
sim_start = "2024-07-15"   # Monday, mid-summer (high load)
sim_end   = "2024-07-22"

sim_data = ami[(ami["timestamp"] >= sim_start) &
               (ami["timestamp"] sort_values("timestamp")

print(f"Simulation period: {sim_start} to {sim_end}")
print(f"Readings to process: {len(sim_data):,}")
print(f"Customers:     {sim_data['customer_id'].nunique():,}")

# Replay data through the adaptive detector
# NOTE: We use itertuples() instead of iterrows() for ~5-10x speed improvement.
# For production-scale streaming, consider a vectorized batch approach or a
# dedicated streaming framework (Kafka + Flink, or pandas pipe with groupby).
results = []
for row in sim_data.itertuples(index=False):
    result = adaptive_detector.ingest(
        customer_id=row.customer_id,
        timestamp=row.timestamp,
        voltage=row.voltage_v,
        energy_kwh=row.energy_kwh,
    )
    if result is not None:
        results.append({
            "customer_id":   row.customer_id,
            "timestamp":  row.timestamp,
            **result,
        })

results_df = pd.DataFrame(results)
detections_df = pd.DataFrame(adaptive_detector.detections)

print(f"\nWindows scored:    {len(results_df):,}")
print(f"Anomalies flagged: {len(detections_df):,}")
print(f"Alert rate:        {len(detections_df)/max(len(results_df),1)*100:.2f}%")

In [ ]:
# Load outage events for the same week
week_outages = outages[
    (outages["fault_detected"] >= sim_start) &
    (outages["fault_detected"] print(f"Actual outage events this week: {len(week_outages)}")
print(week_outages[["fault_detected", "feeder_id", "cause_code",
                    "affected_customers"]].to_string(index=False))

# Timeline visualization: scores and detections overlaid with outage events
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Top: ELBO scores over time (aggregate across customers per hour)
results_df["hour"] = results_df["timestamp"].dt.floor("h")
hourly_max_score = results_df.groupby("hour")["score"].max()
hourly_avg_score = results_df.groupby("hour")["score"].mean()

ax1.fill_between(hourly_avg_score.index, hourly_avg_score.values,
                  alpha=0.3, color="#5FCCDB", label="Avg ELBO score")
ax1.plot(hourly_max_score.index, hourly_max_score.values,
         color="#2D6A7A", linewidth=0.8, alpha=0.7, label="Max ELBO score")

# Mark outage events
for _, outage in week_outages.iterrows():
    ax1.axvline(x=outage["fault_detected"], color="red", linestyle="--",
                alpha=0.8, linewidth=1.5)
# Add a legend entry for outage lines
ax1.plot([], [], color="red", linestyle="--", linewidth=1.5, label="Outage event")

ax1.set_ylabel("ELBO Anomaly Score")
ax1.set_title("Streaming VAE Anomaly Scores — July 15-21, 2024")
ax1.legend(loc="upper right")

# Bottom: detection timeline (heatmap of flagged customers)
if len(detections_df) > 0:
    det_hourly = detections_df.groupby(
        detections_df["timestamp"].dt.floor("h")
    ).size()
    ax2.bar(det_hourly.index, det_hourly.values,
           width=0.04, color="#E53E3E", alpha=0.8, label="Anomaly detections")

for _, outage in week_outages.iterrows():
    ax2.axvline(x=outage["fault_detected"], color="red", linestyle="--",
                alpha=0.8, linewidth=1.5)

ax2.set_xlabel("Time")
ax2.set_ylabel("Detections per Hour")
ax2.set_title("Anomaly Detection Timeline vs Actual Outages")
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis: how many outages had prior anomaly detections?
lead_time_hours = 2  # how far back to look for preceding detections

if len(detections_df) > 0:
    outage_hits = 0
    outage_details = []

    for _, outage in week_outages.iterrows():
        fault_time = outage["fault_detected"]
        window_start = fault_time - pd.Timedelta(hours=lead_time_hours)

        # Check if any detections occurred in the lead-time window
        prior_detections = detections_df[
            (detections_df["timestamp"] >= window_start) &
            (detections_df["timestamp"] len(prior_detections) > 0
        outage_hits += int(hit)
        outage_details.append({
            "fault_time":     fault_time,
            "cause_code":          outage["cause_code"],
            "prior_alerts":   len(prior_detections),
            "detected":       hit,
        })

    correlation_df = pd.DataFrame(outage_details)
    hit_rate = outage_hits / len(week_outages) * 100

    print(f"\nOutage correlation analysis ({lead_time_hours}h lead time):")
    print(f"  Outages with prior anomaly detection: {outage_hits}/{len(week_outages)} ({hit_rate:.1f}%)")
    print(f"\nDetails:")
    print(correlation_df.to_string(index=False))

In [ ]:
import joblib

# Save the trained VAE
torch.save(vae.state_dict(), "voltage_vae.pt")

# Save the fitted scaler (needed to normalize new data consistently)
joblib.dump(scaler, "voltage_scaler.pkl")

# Load:
# vae.load_state_dict(torch.load("voltage_vae.pt"))
# scaler = joblib.load("voltage_scaler.pkl")